# ക്ലൗഡിലെ ഡാറ്റാ സയൻസ്: "Azure ML SDK" വഴി

## പരിചയം

ഈ നോട്ട്‌ബുക്കിൽ, Azure ML ഉപയോഗിച്ച് ഒരു മോഡൽ ട്രെയിൻ ചെയ്യാനും, ഡിപ്ലോയ് ചെയ്യാനും, ഉപയോഗിക്കാനും Azure ML SDK എങ്ങനെ ഉപയോഗിക്കാമെന്ന് നാം പഠിക്കും.

ആവശ്യമായ മുൻ‌പരിചയങ്ങൾ:
1. നിങ്ങൾ ഒരു Azure ML വർക്ക്‌സ്പേസ് സൃഷ്ടിച്ചിട്ടുണ്ട്.
2. നിങ്ങൾ [ഹാർട്ട് ഫെയില്യർ ഡാറ്റാസെറ്റ്](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) Azure ML-ലേക്ക് ലോഡ് ചെയ്തിട്ടുണ്ട്.
3. നിങ്ങൾ ഈ നോട്ട്‌ബുക്ക് Azure ML സ്റ്റുഡിയോയിൽ അപ്‌ലോഡ് ചെയ്തിട്ടുണ്ട്.

അടുത്ത ഘട്ടങ്ങൾ:

1. നിലവിലുള്ള വർക്ക്‌സ്പേസിൽ ഒരു എക്സ്പെരിമെന്റ് സൃഷ്ടിക്കുക.
2. ഒരു കംപ്യൂട്ട് ക്ലസ്റ്റർ സൃഷ്ടിക്കുക.
3. ഡാറ്റാസെറ്റ് ലോഡ് ചെയ്യുക.
4. AutoMLConfig ഉപയോഗിച്ച് AutoML കോൺഫിഗർ ചെയ്യുക.
5. AutoML എക്സ്പെരിമെന്റ് റൺ ചെയ്യുക.
6. ഫലങ്ങൾ പരിശോധിച്ച് മികച്ച മോഡൽ കണ്ടെത്തുക.
7. മികച്ച മോഡൽ രജിസ്റ്റർ ചെയ്യുക.
8. മികച്ച മോഡൽ ഡിപ്ലോയ് ചെയ്യുക.
9. എന്റ്പോയിന്റ് ഉപയോഗിക്കുക.

## Azure മെഷീൻ ലേണിംഗ് SDK-നു പ്രത്യേകമായ ഇമ്പോർട്ടുകൾ


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Initialize Workspace
സ്ഥിരീകരിച്ച കോൺഫിഗറേഷൻ നിന്ന് ഒരു വർക്ക്‌സ്പേസ് ഒബ്ജക്റ്റ് ആരംഭിക്കുക. .\config.json എന്ന സ്ഥലത്ത് കോൺഫിഗ് ഫയൽ ഉണ്ടെന്ന് ഉറപ്പാക്കുക.


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create an Azure ML experiment

നാം ഇപ്പോൾ ആരംഭിച്ച വർക്ക്സ്പേസിൽ 'aml-experiment' എന്ന പേരിൽ ഒരു പരീക്ഷണം സൃഷ്ടിക്കാം.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## കംപ്യൂട്ട് ക്ലസ്റ്റർ സൃഷ്ടിക്കുക
നിങ്ങളുടെ AutoML റൺക്കായി ഒരു [compute target](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) സൃഷ്ടിക്കേണ്ടതുണ്ട്.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Data
നിങ്ങൾ ഡാറ്റാസെറ്റ് Azure ML-ലേക്ക് അപ്‌ലോഡ് ചെയ്തിട്ടുണ്ടെന്ന് ഉറപ്പാക്കുക, കൂടാതെ കീ ഡാറ്റാസെറ്റിന്റെ പേരുമായി ഒരേ പേരിലാണ്.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## ഓട്ടോഎംഎൽ കോൺഫിഗറേഷൻ


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## ഓട്ടോഎംഎൽ റൺ


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

## മികച്ച മോഡൽ സംരക്ഷിക്കുക


In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## മികച്ച മോഡൽ വിന്യസിക്കുക

മികച്ച മോഡൽ വിന്യസിക്കാൻ താഴെ കൊടുത്തിരിക്കുന്ന കോഡ് പ്രവർത്തിപ്പിക്കുക. Azure ML പോർട്ടലിൽ വിന്യസനത്തിന്റെ നില നിങ്ങൾക്ക് കാണാം. ഈ ഘട്ടം കുറച്ച് മിനിറ്റുകൾ എടുക്കാം.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## എന്റ്പോയിന്റ് ഉപയോഗിക്കുക
താഴെ കൊടുത്തിരിക്കുന്ന ഇൻപുട്ട് സാമ്പിളിൽ നിങ്ങൾക്ക് ഇൻപുട്ടുകൾ ചേർക്കാം.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചെങ്കിലും, സ്വയം പ്രവർത്തിക്കുന്ന വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖയാണ് പ്രാമാണികമായ ഉറവിടം എന്ന് പരിഗണിക്കേണ്ടതാണ്. നിർണായക വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
